In [39]:
%config Completer.use_jedi = False  # enable code auto-completion

import numpy as np    # library for numerical computations (vectors, matrices, tensors)
import pandas as pd    # library for data manipulation and analysis
import matplotlib.pyplot as plt    # library providing tools for plotting data
from sklearn.preprocessing import PolynomialFeatures    # function to generate polynomial and interaction features
from sklearn.linear_model import LinearRegression, HuberRegressor    # classes providing Linear Regression with ordinary squared error loss and Huber loss, respectively
from sklearn.metrics import mean_squared_error    # function to calculate mean squared error 
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
#import all the usefull packages
from pyproj import CRS
import os
from geopandas.tools import geocode
import networkx as nx
from shapely.geometry import Point, LineString, Polygon
from geopandas.tools import geocode
import folium
import requests


In [40]:
data = pd.read_csv('orders_autumn_2020.csv')
print(data.columns)
print(len(data))
data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'])
data['dayOfWeek'] = data['TIMESTAMP'].dt.day_name()
data['date'] =''
for i in data['dayOfWeek']:
    if i == 'Saturday' or i == 'Sunday':
        data['date'] = 'weekend'
    else:
        data['date'] = 'weekday'
data = data[['TIMESTAMP',  'ESTIMATED_DELIVERY_MINUTES' ,'ACTUAL_DELIVERY_MINUTES', 'dayOfWeek', 'date' ]]
data['date'] = pd.get_dummies(data["date"])
data.set_index(pd.DatetimeIndex(data['TIMESTAMP']), inplace=True)
data = data.between_time('10:00','13:00')

print(len(data))

Index(['TIMESTAMP', 'ACTUAL_DELIVERY_MINUTES - ESTIMATED_DELIVERY_MINUTES',
       'ITEM_COUNT', 'USER_LAT', 'USER_LONG', 'VENUE_LAT', 'VENUE_LONG',
       'ESTIMATED_DELIVERY_MINUTES', 'ACTUAL_DELIVERY_MINUTES',
       'CLOUD_COVERAGE', 'TEMPERATURE', 'WIND_SPEED', 'PRECIPITATION'],
      dtype='object')
18706
4137


In [41]:
X = data[['ESTIMATED_DELIVERY_MINUTES', 'date']].to_numpy().reshape(-1, 2)
y = data['ACTUAL_DELIVERY_MINUTES'].to_numpy()

In [42]:
from sklearn.model_selection import train_test_split 

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)
print(y_train.shape)

(2771,)


In [43]:
degrees = [1,2, 3, 4, 5,6,7,8,9, 10]    

# we will use this variable to store the resulting training errors for each polynomial degree
tr_errors = []          
val_errors = []

plt.figure(figsize=(8, 20))    # create a new figure with size 8*20
for i, degree in enumerate(degrees):    # use for-loop to fit polynomial regression models with different degrees    
    lin_regr = LinearRegression(fit_intercept=False) # NOTE: "fit_intercept=False" as we already have a constant iterm in the new feature X_poly 
    poly = PolynomialFeatures(degree=degree)    # generate polynomial features
    X_train_poly = poly.fit_transform(X_train)    # fit and transform the raw features
    lin_regr.fit(X_train_poly, y_train)    # apply linear regression to these new features and labels
    y_pred_train = lin_regr.predict(X_train_poly)
    tr_error = mean_squared_error(y_train, y_pred_train)   
    print(f' traing error: {tr_error}')
    X_val_poly =  poly.transform(X_val)   
    y_pred_val = lin_regr.predict(X_val_poly)   
    val_error = mean_squared_error(y_val, y_pred_val)
    print(f' validation error: {val_error}')
    tr_errors.append(tr_error)
    val_errors.append(val_error)
    X_fit = np.linspace(-25, 25, 100)

 traing error: 72.87724812836993
 validation error: 63.544998023729896
 traing error: 72.66312351969364
 validation error: 63.466303570403234
 traing error: 72.55567239337473
 validation error: 63.22769351006356
 traing error: 72.53544585121725
 validation error: 63.25365098304113
 traing error: 72.53386990145498
 validation error: 63.26326658760604
 traing error: 72.51507644187501
 validation error: 63.940340786458954
 traing error: 72.51443584557741
 validation error: 64.38741913850802
 traing error: 72.51890785746451
 validation error: 63.322956839096015
 traing error: 72.52389589895085
 validation error: 63.256333069743825
 traing error: 72.53241241434256
 validation error: 63.312890567018044


<Figure size 576x1440 with 0 Axes>